# 欠損値を穴埋めする関数を作成

## 条件

- 欠損箇所を埋める既定値には列ごとにある
- データはintとflortの場合がある
- 列内のNoneのところだけに穴埋め処理し、他は影響なし
- 空文字はNone扱い
- 既定値の値もデータ型もそのまま変えずに穴埋め
- 穴埋めの既定値を基本的にint型として、float型で穴埋めしたい列名を引数でリスト形式で指定できる

## 困りごと

pd.fillna(var)を使うと、以下の機能が実現できない。

- 欠損値を既定値で埋めるけど、他の値のデータ型は壊したくない。
- 列ごとに別の値で埋めたい。
- NaNは無視したい。




## データフレームを作成

In [47]:
# 欠損値の穴埋め
import numpy as np
import pandas as pd
dfm = pd.DataFrame(
    [
    {"hash":10,"age":28,"wt":56.1},
    {"hash":20,"age":None,"wt":None},
    {"hash":30,"age":'',"wt":''},
    {"hash":40,"name":33,"wt":100}
    ],columns=["hash","age","wt"]
)
dfm



,hash,age,wt
0,10,28,56.1
1,20,None,None
2,30,,
3,40,NaN,100


## 各データのデータ型を確認

In [48]:
for col in dfm.columns:
    print('[{}]=================='.format(col))
    print(dfm[col].map(type))
    print()

[hash]==================
0    <class 'int'>
1    <class 'int'>
2    <class 'int'>
3    <class 'int'>
Name: hash, dtype: object

[age]==================
0         <class 'int'>
1    <class 'NoneType'>
2         <class 'str'>
3       <class 'float'>
Name: age, dtype: object

[wt]==================
0       <class 'float'>
1    <class 'NoneType'>
2         <class 'str'>
3         <class 'int'>
Name: wt, dtype: object



## 空文字をNoneに置換

In [49]:
print(type(''))
print(type(None))
print(type(np.nan))

<class 'str'>
<class 'NoneType'>
<class 'float'>


In [50]:
# 空文字はNone扱い
dfm = dfm.replace('', None)
dfm


,hash,age,wt
0,10,28,56.1
1,20,None,None
2,30,None,None
3,40,NaN,100


In [51]:
for col in dfm.columns:
    print('[{}]=================='.format(col))
    print(dfm[col].map(type))
    print()

[hash]==================
0    <class 'int'>
1    <class 'int'>
2    <class 'int'>
3    <class 'int'>
Name: hash, dtype: object

[age]==================
0         <class 'int'>
1    <class 'NoneType'>
2    <class 'NoneType'>
3       <class 'float'>
Name: age, dtype: object

[wt]==================
0       <class 'float'>
1    <class 'NoneType'>
2    <class 'NoneType'>
3         <class 'int'>
Name: wt, dtype: object



## デフォルト値を設定

In [52]:
HASH_DEFAULT = 99
AGE_DEFAULT = 99 # int型
WT_DEFAULT = 99.9 # float型

In [53]:
x_int = 99
type(x_int)

int

In [54]:
x_float = float(x_int)
type(x_float)

float

In [55]:
x_float

99.0

In [56]:
type(type(x_float))

type

## データ型を変えずに穴埋め

1. データ型を保持しておくデータフレームを作成
2. データをNoneにのみ代入（NaNは無視）
3. データ型が代入した値に応じて列ごとに変化してしまうので、

In [57]:
# 1. データ型を保持しておくデータフレームを作成
dfm_type = dfm.copy()

# データ型で置き代えるラムダ式
# set_type = lambda x: type(x)
def get_type(val):
    return type(val)

dfm_type = dfm_type.applymap(get_type)

dfm_type

,hash,age,wt
0,<class 'int'>,<class 'int'>,<class 'float'>
1,<class 'int'>,<class 'NoneType'>,<class 'NoneType'>
2,<class 'int'>,<class 'NoneType'>,<class 'NoneType'>
3,<class 'int'>,<class 'float'>,<class 'int'>


In [58]:
# 2. データ型を確認し、'NoneType'の場合のみ代入

DEFAULT = AGE_DEFAULT

def set_default(val):
    if isinstance(val, type(None)):
        return DEFAULT
    else:
        return val    
    
    
dfm['age'] = dfm['age'].map(set_default)

# 保持しておいたデータ型を適用
# うまくいかない→列ごとに制約がかかっている…？
for i in range(len(dfm)):
    pre_type = dfm_type.loc[i, 'age']
    if pre_type == type(None):
        if isinstance(AGE_DEFAULT, int):
            dfm.loc[i, 'age'] = int(dfm.loc[i, 'age'])
        elif isinstance(AGE_DEFAULT, float):
            dfm.loc[i, 'age'] = float(dfm.loc[i, 'age'])
    elif pre_type == int:
        dfm.loc[i, 'age'] = int(dfm.loc[i, 'age'])
    elif pre_type == float:
        dfm.loc[i, 'age'] = float(dfm.loc[i, 'age'])

dfm

,hash,age,wt
0,10,28.0,56.1
1,20,99.0,None
2,30,99.0,None
3,40,NaN,100


In [59]:
AGE_DEFAULT

99

In [60]:
dfm_type.loc[i, 'age']

float

In [61]:
for col in dfm.columns:
    print('[{}]=================='.format(col))
    print(dfm[col].map(type))
    print()

[hash]==================
0    <class 'int'>
1    <class 'int'>
2    <class 'int'>
3    <class 'int'>
Name: hash, dtype: object

[age]==================
0    <class 'float'>
1    <class 'float'>
2    <class 'float'>
3    <class 'float'>
Name: age, dtype: object

[wt]==================
0       <class 'float'>
1    <class 'NoneType'>
2    <class 'NoneType'>
3         <class 'int'>
Name: wt, dtype: object



In [39]:
int(99.0)

99

---

In [9]:
# 欠損箇所を埋める既定値には列ごとにある。データはintとflortの場合がある
# これはダメなケース
# AGE_DEFAULT = 99
# WT_DEFAULT = 99.9

dfm = dfm.fillna(99)

dfm

,hash,age,wt
0,10,28,56.1
1,20,99,99.0
2,30,99,99.0
3,40,99,100.0


In [39]:
# 2. 列ごとに異なる値で穴埋め
# https://note.nkmk.me/python-pandas-nan-dropna-fillna/

dfm = dfm.fillna({
    'age': AGE_DEFAULT, 
    'wt': WT_DEFAULT
    })

dfm

,hash,age,wt
0,10,28,56.1
1,20,99,99.9
2,30,99,99.9
3,40,99,100.0


In [ ]:
# 仕様検討中
# 　既定値の値もデータ型もそのまま変えずに穴埋めをしたい。
# 　穴埋めの既定値を基本的にint型として、float型で穴埋めしたい列名はリストにして引数で指定する
#　列内のNoneのところだけに穴埋め処理し、他は影響しないようにしたい


## 機能を一般化